In [35]:
import pandas as pd
from gensim.models import Word2Vec
import ast

# Bước 1: Đọc dữ liệu đầu vào
# Điều chỉnh tham số 'sep' nếu cần thiết dựa trên định dạng file CSV của bạn
goodware_train = pd.read_csv(
    'input/b/b_good_train.csv',
    sep=';',  # Sử dụng dấu chấm phẩy làm phân cách
    header=None,
    names=['api_sequence', 'label']  # Đặt tên cho hai cột
)
malware_train = pd.read_csv('input/b/b_mal_train.csv', sep=';', header=None, names=['api_sequence', 'label'])


def parse_sequence(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        # Nếu không phải là danh sách, chuyển đổi thành danh sách chứa chuỗi đó
        return [x]

# Bước 2: Xử lý các sequences
# Điều chỉnh hàm split nếu các API calls được phân tách bằng ký tự khác
# goodware_sequences = goodware_train['api_sequence'].apply(lambda x: str(x).split(','))
# malware_sequences = malware_train['api_sequence'].apply(lambda x: str(x).split(','))

goodware_train['api_sequence'] = goodware_train['api_sequence'].apply(parse_sequence)
malware_train['api_sequence'] = malware_train['api_sequence'].apply(parse_sequence)

print("Goodware training data sample:")
print(goodware_train['api_sequence'].head())

print("\nMalware training data sample:")
print(malware_train['api_sequence'].head())

# Bước 3: Kết hợp các sequences
# all_sequences = list(goodware_sequences) + list(malware_sequences)

all_sequences = goodware_train['api_sequence'].tolist() + malware_train['api_sequence'].tolist()

# Bước 4: Huấn luyện mô hình Word2Vec
model = Word2Vec(sentences=all_sequences, vector_size=250, window=6, min_count=1, workers=6)

model.save('./b_word2vec.model')

Goodware training data sample:
0                                            [content]
1    [RegCloseKey, RegCreateKeyExA, RegDeleteKeyA, ...
2    [CoInitialize, CoCreateInstance, CoUninitializ...
3                                        [_CorExeMain]
4    [LoadLibraryA, GetProcAddress, VirtualProtect,...
Name: api_sequence, dtype: object

Malware training data sample:
0                                            [content]
1    [GetSystemDirectoryA, IsDBCSLeadByte, LocalAll...
2    [GetSystemDirectoryA, IsDBCSLeadByte, LocalAll...
3    [GetSystemDirectoryA, IsDBCSLeadByte, LocalAll...
4    [GetSystemDirectoryA, IsDBCSLeadByte, LocalAll...
Name: api_sequence, dtype: object


In [36]:
from sklearn.cluster import KMeans
import pickle
import numpy as np
import pandas as pd

# Bước 5: Lấy danh sách các API functions
api_functions = model.wv.index_to_key

# Bước 6: Lấy embeddings cho các API functions
api_embeddings = np.array([model.wv[api] for api in api_functions])

# Bước 7: Đặt số lượng clusters là 10
n_clusters = 10

# Bước 8: Áp dụng K-Means clustering trên các API embeddings
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(api_embeddings)

# Bước 9: Gán các API functions vào clusters
clusters = {i: [] for i in range(n_clusters)}
for idx, label in enumerate(kmeans.labels_):
    clusters[label].append(api_functions[idx])

# Bước 10: Xuất kết quả n_clusters và clusters[]
print(f"Số lượng clusters (n_clusters): {n_clusters}")
print("Danh sách các API functions trong từng cluster:")
for cluster_id in clusters:
    print(f"\nCluster {cluster_id}:")
    for api in clusters[cluster_id]:
        print(f" - {api}")

# (Tùy chọn) Lưu kết quả clustering vào file
with open('api_clusters.pkl', 'wb') as f:
    pickle.dump(clusters, f)
    

with open('api_clusters.pkl', 'wb') as f:
    pickle.dump(clusters, f)

# Bước 11: Chuyển đổi kết quả clustering thành file CSV
clusters_df = pd.DataFrame({
    'cluster': [f'cluster {k}' for k in clusters.keys()],
    'content': [v for v in clusters.values()]
})

# Lưu DataFrame thành file CSV với dấu phân cách là dấu chấm phẩy
clusters_df.to_csv('clusters.csv', sep=';', index=False)

print("Đã lưu kết quả clustering vào 'clusters.csv'")

Số lượng clusters (n_clusters): 10
Danh sách các API functions trong từng cluster:

Cluster 0:
 - HeapLock
 - GetTempFileNameA
 - GetCurrentDirectoryW
 - OpenWindowStationW
 - GetFileVersionInfoA
 - LoadKeyboardLayoutW
 - LoadKeyboardLayoutA
 - SignalObjectAndWait
 - _adjust_fdiv
 - _llseek
 - CloseThreadpoolTimer
 - WaitForThreadpoolTimerCallbacks
 - MessageBoxIndirectW
 - EnumDesktopWindows
 - wvsprintfW
 - CoFreeUnusedLibraries
 - __clean_type_info_names_internal
 - __CppXcptFilter
 - DialogBoxIndirectParamW
 - TraceEvent
 - CreateHalftonePalette
 - RegFlushKey
 - EnumSystemLocalesW
 - EventActivityIdControl
 - CreateFontA
 - GetWindowTextLengthA
 - ProgIDFromCLSID
 - InsertMenuW
 - IsCharAlphaNumericW
 - OemToCharBuffA
 - GlobalGetAtomNameA
 - CreateThreadpoolWait
 - SetThreadpoolWait
 - CreateProcessAsUserW
 - RemovePropA
 - _calloc_crt
 - WaitForThreadpoolWorkCallbacks
 - CloseThreadpoolWait
 - IsValidSecurityDescriptor
 - CreateDialogIndirectParamA
 - __crtTerminateProcess
 - __